In [ ]:
from __future__ import print_function

import math

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from pytest import approx

from fastimgproto.fixtures.image import (
    add_gaussian2d_to_image,
    gaussian_point_source,
)
from fastimgproto.sourcefind.fit import Gaussian2dParams
from fastimgproto.sourcefind.image import SourceFindImage, _STRUCTURE_ELEMENT

from fastimgproto.bindings.sourcefind import (
    cpp_sourcefind,
    cpp_sourcefind_result_to_islandparams,
)

from fastimgproto.fixtures.sourcefind import (
    generate_random_source_params)

import logging

logger = logging.getLogger(__name__)

In [ ]:
%matplotlib inline

# Plot image pixels in cartesian ordering (i.e. y-positive == upwards):
plt.rcParams['image.origin'] = 'lower'
# Make plots bigger
plt.rcParams['figure.figsize'] = 10, 10

In [ ]:
ydim = 64
xdim = 32
image_shape = (ydim, xdim)


In [ ]:
detection_n_sigma=5.
analysis_n_sigma=3.
rms_est=1.

In [ ]:
test_source = Gaussian2dParams(x_centre=18.48820770355941, y_centre=34.05960466345559, amplitude=12.609448034550804,
                               semimajor=2.913573230793465, semiminor=0.7230940150989847, theta=-1.4507887213254516)


img = np.zeros(image_shape)
add_gaussian2d_to_image(test_source, img)

In [ ]:
img.max()

In [ ]:
sfimg =SourceFindImage(img, detection_n_sigma=4., analysis_n_sigma=3., rms_est=1.,find_negative_sources=True)
len(sfimg.islands)

In [ ]:
sfimg.fit_gaussian_2d(sfimg.islands[0],verbose=0)

In [ ]:
import stp_python
ceres_diffmethod = stp_python.CeresDiffMethod.AnalyticDiff
ceres_solvertype = stp_python.CeresSolverType.TrustRegion_DenseQR
median_method = stp_python.MedianMethod.ZEROMEDIAN
stp_pars = dict(image_data=np.asfortranarray(img),
            detection_n_sigma=detection_n_sigma,
            analysis_n_sigma=analysis_n_sigma,
            rms_est=rms_est,
            find_negative_sources=False,
            sigma_clip_iters=0,
            median_method=median_method,
            gaussian_fitting=False,
            generate_labelmap=False,
            ceres_diffmethod=ceres_diffmethod,
            ceres_solvertype=ceres_solvertype)

result = stp_python.source_find_wrapper(**stp_pars)
result =[cpp_sourcefind_result_to_islandparams(r) for r in result]
result

In [ ]:
pars = sfimg.islands[0].params

In [ ]:
pars.fitter_report.success

In [ ]:
pars.moments_fit

In [ ]:
pars.leastsq_fit

In [ ]:
test_source

In [ ]:
leastsq_img = np.zeros(image_shape)
add_gaussian2d_to_image(pars.leastsq_fit, leastsq_img)

In [ ]:
# fig = plt.figure(figsize=(8, 10))
plt.imshow(img- leastsq_img)
plt.colorbar()
plt.scatter(test_source.x_centre, test_source.y_centre)
peak_idx =sfimg.islands[0].extremum.index
plt.scatter(peak_idx.x, peak_idx.y)

In [ ]:
binary_map = sfimg.data > sfimg.analysis_n_sigma * sfimg.rms_est

In [ ]:
plt.imshow(binary_map)

In [ ]:
label_map, n_labels = ndimage.label(binary_map, structure=_STRUCTURE_ELEMENT)

In [ ]:
n_labels

In [ ]:
range(1, n_labels + 1)

In [ ]:
plt.imshow(label_map)

In [ ]:
ndimage.maximum(sfimg.data, label_map, index=range(1, n_labels + 1))

In [ ]:
plt.imshow(sfimg.islands[0].data)

In [ ]:
# plt.imshow(sfimg.islands[1].data)